In [1]:
import os
import numpy as np
import pandas as pd
import pyreadstat
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.mixture import GaussianMixture
from joblib import Parallel, delayed
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans as SparkKMeans, GaussianMixture as SparkGMM
from pyspark.ml.feature import VectorAssembler, PCA as SparkPCA
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vectors

In [2]:
# load the path
os.chdir('/Users/lijiayu/Desktop/MACS_30700_Final/')

In [3]:
df = pd.read_spss('dataset/EU/gender_norm.sav')

In [5]:
# define column mapping to match pokec attributes
eurobarometer_mapping = {
    "d10": "gender",
    "d11": "age",
    "d8r2": "age_finishing_education",
    "d7r1": "marital_status",
    "d7": "children_status"
}

# select only relevant columns and rename them
df_selected = df[list(eurobarometer_mapping.keys())].rename(columns=eurobarometer_mapping)

# display the first few rows to verify selection
display(df_selected.head())

,gender,age,age_finishing_education,marital_status,children_status
0,Man,22.0,Still studying,Other (SPONT.),Other (SPONT.)
1,Woman,24.0,Still studying,Other (SPONT.),Other (SPONT.)
2,Woman,63.0,16-19 years,(Re-)Married (1-4 in d7),(Re-)Married: living without children
3,Man,63.0,16-19 years,Single living with partner (5-8 in d7),Single living w partner: without children
4,Woman,81.0,Up to 15 years,(Re-)Married (1-4 in d7),(Re-)Married: living without children


In [6]:
# redefine the mapping for marital status to match pokec categories
marital_status_mapping = {
    "Other (SPONT.)": np.nan,
    "(Re-)Married (1-4 in d7)": "Married",
    "Single living with partner (5-8 in d7)": "Cohabiting",
    "Widow (13-14 in d7)": "Divorced or Widowed",
    "Single (9-10 in d7)": "Single",
    "Divorced or separated (11-12 in d7)": "Divorced or Widowed"
}

# apply mapping to marital status
df_selected["marital_status"] = df_selected["marital_status"].astype(str).replace(marital_status_mapping)
df_selected["marital_status"] = df_selected["marital_status"].replace("nan", np.nan).astype("category")

# verify unique values after mapping
print(df_selected["marital_status"].unique())

[NaN, 'Married', 'Cohabiting', 'Divorced or Widowed', 'Single']
Categories (4, object): ['Cohabiting', 'Divorced or Widowed', 'Married', 'Single']


In [11]:
# correct mapping for children_status
children_mapping = {
    "(Re-)Married: w children of this marriage": "Yes",
    "(Re-)Married: w children of this and previous marriage": "Yes",
    "(Re-)Married: w children of previous marriage": "Yes",
    "Single living w partner: w children of this union": "Yes",
    "Single living w partner: w children of previous union": "Yes",
    "Single living w partner: w children of this and previous union": "Yes",
    "Single: with children": "Yes",
    "Divorced/Separated: living with children": "Yes",
    "Widow/er: living with children": "Yes",
    "(Re-)Married: living without children": "No",
    "Single living w partner: without children": "No",
    "Single: living without children": "No",
    "Divorced/Separated: living without children": "No",
    "Widow/er: living without children": "No",
    np.nan: np.nan  # keep NaN values unchanged
}

# apply mapping
df_selected["children_status"] = df_selected["children_status"].replace(children_mapping).astype("category")

# check unique values after mapping
print(df_selected["children_status"].unique())

[NaN, 'No', 'Yes']
Categories (2, object): ['No', 'Yes']


In [8]:
# standardize gender values to match Pokec
gender_mapping = {
    "Man": "Male",
    "Woman": "Female"
}

df_selected["gender"] = df_selected["gender"].replace(gender_mapping).astype("category")

# verify unique values after mapping
print(df_selected["gender"].unique())

['Male', 'Female']
Categories (2, object): ['Female', 'Male']


In [12]:
# ensure age is numeric
df_selected["age"] = pd.to_numeric(df_selected["age"], errors="coerce")

# verify conversion
print(df_selected["age"].dtype)  # should be 'float64' or 'int64'

float64


In [13]:
# redefine the mapping for education level
education_mapping = {
    "Up to 15 years": "Low",
    "16-19 years": "Middle",
    "20 years and older": "High",
    "No full-time education": "No education",
    "Still studying": None  # Will be reassigned based on age
}

# apply education mapping
df_selected["education_level"] = df_selected["age_finishing_education"].replace(education_mapping)

# handle "Still Studying" cases by assigning education level based on age
df_selected.loc[df_selected["age_finishing_education"] == "Still studying", "education_level"] = df_selected["age"].apply(
    lambda x: "Low" if pd.notna(x) and x <= 15 else 
              "Middle" if pd.notna(x) and 16 <= x <= 19 else 
              "High" if pd.notna(x) and x > 19 else np.nan
)

# ensure column is categorical
df_selected["education_level"] = df_selected["education_level"].astype("category")

# verify unique values after mapping
print(df_selected["education_level"].unique())


['High', 'Middle', 'Low', NaN, 'No education']
Categories (4, object): ['High', 'Low', 'Middle', 'No education']


In [14]:
# define age groups to match Pokec dataset
age_bins = [15, 18, 25, 44, 64, np.inf]
age_labels = ["15-18", "19-25", "26-44", "45-64", "65+"]

df_selected["age_group"] = pd.cut(df_selected["age"], bins=age_bins, labels=age_labels, right=True).astype("category")

# verify unique values after mapping
print(df_selected["age_group"].unique())

['19-25', '45-64', '65+', '26-44', '15-18', NaN]
Categories (5, object): ['15-18' < '19-25' < '26-44' < '45-64' < '65+']


In [17]:
# display first few rows of df_selected
display(df_selected.head())

# display unique values for each categorical column in a structured format
for col in ["gender", "age_group", "education_level", "marital_status", "children_status"]:
    print(f"\nUnique values for {col}:")
    display(df_selected[col].value_counts(dropna=False))  # shows frequency of each unique value

,gender,age,age_finishing_education,marital_status,children_status,education_level,age_group
0,Male,22.0,Still studying,NaN,NaN,High,19-25
1,Female,24.0,Still studying,NaN,NaN,High,19-25
2,Female,63.0,16-19 years,Married,No,Middle,45-64
3,Male,63.0,16-19 years,Cohabiting,No,Middle,45-64
4,Female,81.0,Up to 15 years,Married,No,Low,65+



Unique values for gender:


Female    15284
Male      12517
Name: gender, dtype: int64


Unique values for age_group:


45-64    9583
26-44    7864
65+      7234
19-25    2243
15-18     704
NaN       173
Name: age_group, dtype: int64

...

Middle          12696
High            10036
Low              4264
NaN               602
No education      203
Name: education_level, dtype: int64


Unique values for marital_status:


Married                14737
Divorced or Widowed     5016
Single                  4674
Cohabiting              3103
NaN                      271
Name: marital_status, dtype: int64


Unique values for children_status:


No     17160
Yes    10370
NaN      271
Name: children_status, dtype: int64

In [18]:
# drop rows where any of the key sociodemographic attributes are NaN
df_selected = df_selected.dropna(subset=["gender", "age_group", "education_level", "marital_status", "children_status"])

# check final dataset size
print(f"Final dataset size after removing NaN values: {df_selected.shape[0]}")

Final dataset size after removing NaN values: 26955


In [19]:
# check for missing values across all columns in df_selected
missing_values = df_selected.isna().sum()
missing_values = missing_values[missing_values > 0]  # filter only columns with missing values

# display missing values per column
if missing_values.empty:
    print("No missing values in df_selected.")
else:
    print("Missing values detected:")
    print(missing_values)

No missing values in df_selected.


In [20]:
# export df_selected as eu.csv
df_selected.to_csv("dataset/EU/eu.csv", index=False)

print("Export completed: dataset/EU/eu.csv")


Export completed: dataset/EU/eu.csv
